In [1]:
import sys, os, time, datetime    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.webdriver.common.by import By

In [2]:
#크롬 웹브라우저 실행
driver = webdriver.Chrome()
url = "https://finance.naver.com/sise/theme.nhn?&page=1"
driver.get(url)
time.sleep(2)

In [ ]:
# 7페이지 for문으로 반복
for i in range(7):
    html = requests.get('https://finance.naver.com/sise/theme.nhn?&page={}'.format(i + 1))
    table = pd.read_html(html.text)
    if i == 0:
        stock = table[0].dropna().reset_index(drop = True)
    else:
        stock1 = table[0].dropna().reset_index(drop = True)
        stock = pd.concat([stock, stock1])

stock = stock.reset_index(drop = True)
stock.drop(stock.columns[1:], axis=1, inplace=True) ### 첫째열 제외하고 삭제
stock = stock.set_axis(['theme'], axis=1) ### 칼럼 이름 변경
pd.options.display.max_rows = 4000
print(stock)

In [38]:
### 1페이지부터 7페이지까지 모두 이동하면서 모든 행을 클릭해서 테마이름별 No 정보 취함
def theme_url_load(page):
    url = ('https://finance.naver.com/sise/theme.nhn?&page={}'.format(page))
    driver.get(url)
    time.sleep(1)
k=1
l_data = []

first_url = "https://finance.naver.com/sise/theme.naver"
driver.get(first_url)
print('테마개수 : {0}'.format(len(stock)))

for i in range(1, 7):
    theme_url_load(i)
    for j in range(1,40):
        if k <= len(stock):
            print(i, j, k)
            quotient, remainder = divmod(j, 5)
            if remainder == 0:
                quotient-=1
            table_row = j + (quotient+1) * 3    
                        
            xpath = ('//*[@id="contentarea_left"]/table[1]/tbody/tr[{}]/td[1]/a'.format(table_row))
            print(table_row, xpath)
            
            clickable = driver.find_element(By.XPATH, xpath)
            ActionChains(driver)\
                .click(clickable)\
                .perform()
            
            url = driver.current_url ### 페이지의 URL을 받음           
            html = requests.get(url)
            # using the BeautifulSoup module
            soup = BeautifulSoup(html.text, 'html.parser')

            for title in soup.find_all('title'):
                str_title = title.get_text().strip()
            x = str_title.find(':')
            str_title = str_title[:x-1]
            print(str_title, url)
            l_data.append([str_title, url])
            k += 1
            driver.back()
            time.sleep(1)

print(l_data)  

테마개수 : 279
1 1 1
4 //*[@id="contentarea_left"]/table[1]/tbody/tr[4]/td[1]/a
야놀자(Yanolja) https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=483
1 2 2
5 //*[@id="contentarea_left"]/table[1]/tbody/tr[5]/td[1]/a
인터넷 대표주 https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=49
1 3 3
6 //*[@id="contentarea_left"]/table[1]/tbody/tr[6]/td[1]/a
일자리(취업) https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=324
1 4 4
7 //*[@id="contentarea_left"]/table[1]/tbody/tr[7]/td[1]/a
가상화폐(비트코인 등) https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=331
1 5 5
8 //*[@id="contentarea_left"]/table[1]/tbody/tr[8]/td[1]/a
건설 대표주 https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=154
1 6 6
12 //*[@id="contentarea_left"]/table[1]/tbody/tr[12]/td[1]/a
항공/저가 항공사(LCC) https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=166
1 7 7
13 //*[@id="contentarea_left"]/table[1]/tbody/tr[13]/td[1]/a
홈쇼핑 https://finance.naver.com

In [43]:
path = 'C:/Users/Admin/py_stock_analysis/'

df = pd.DataFrame(l_data)
df.to_csv(path+'thema_url.csv', encoding='utf-8', index=False)

In [72]:
columns = ['theme', 'url']
df = pd.read_csv(path+'thema_url.csv', names=columns, encoding='utf-8', header=None, skiprows=1)

df['theme_no'] = df['url'].str[69:]
df.head()

,theme,url,theme_no
0,야놀자(Yanolja),https://finance.naver.com/sise/sise_group_deta...,483
1,인터넷 대표주,https://finance.naver.com/sise/sise_group_deta...,49
2,일자리(취업),https://finance.naver.com/sise/sise_group_deta...,324
3,가상화폐(비트코인 등),https://finance.naver.com/sise/sise_group_deta...,331
4,건설 대표주,https://finance.naver.com/sise/sise_group_deta...,154


In [96]:
import urllib
l_data = []
for d in df.iterrows():
    theme = d[1][0]
    theme_no = d[1][2]
    url = urllib.parse.quote_plus(d[1][1], "/:?=&") # need this line
    driver.get(url)
    html = requests.get(url)
    # using the BeautifulSoup module
    soup = BeautifulSoup(html.text, 'html.parser')

    for row in soup.select('tbody tr'):
        row_text = [x.text for x in row.find_all('td')]
        if row_text[0] != '':
            if row_text[0][-1:] == '*':
                market = 'kosdaq'
            else: market = 'kospi'
            print(theme, theme_no, market, row_text[0])
            l_data.append([theme, theme_no, market, row_text[0]])
df = pd.DataFrame(l_data)
df.to_csv(path+'thema_stock.csv', encoding='utf-8', index=False)

야놀자(Yanolja) 483 kospi 한화투자증권 
야놀자(Yanolja) 483 kosdaq SBI인베스트먼트 *
야놀자(Yanolja) 483 kospi 한화생명 
야놀자(Yanolja) 483 kosdaq 아주IB투자 *
야놀자(Yanolja) 483 kosdaq 그래디언트 *
야놀자(Yanolja) 483 kospi 
야놀자(Yanolja) 483 kospi 
인터넷 대표주 49 kospi 카카오 
인터넷 대표주 49 kospi NAVER 
인터넷 대표주 49 kospi 
인터넷 대표주 49 kospi 
일자리(취업) 324 kosdaq 오픈놀 *
일자리(취업) 324 kosdaq 에스코넥 *
일자리(취업) 324 kospi 윌비스 
일자리(취업) 324 kosdaq TS인베스트먼트 *
일자리(취업) 324 kosdaq 대성창투 *
일자리(취업) 324 kosdaq 원티드랩 *
일자리(취업) 324 kosdaq 사람인 *
일자리(취업) 324 kosdaq DSC인베스트먼트 *
일자리(취업) 324 kosdaq 아이비김영 *
일자리(취업) 324 kosdaq 메가엠디 *
일자리(취업) 324 kospi 
일자리(취업) 324 kospi 
가상화폐(비트코인 등) 331 kosdaq 위지트 *
가상화폐(비트코인 등) 331 kosdaq 위메이드 *
가상화폐(비트코인 등) 331 kosdaq 우리기술투자 *
가상화폐(비트코인 등) 331 kosdaq 갤럭시아머니트리 *
가상화폐(비트코인 등) 331 kospi 카카오 
가상화폐(비트코인 등) 331 kosdaq 한일진공 *
가상화폐(비트코인 등) 331 kosdaq SBI인베스트먼트 *
가상화폐(비트코인 등) 331 kosdaq 다날 *
가상화폐(비트코인 등) 331 kosdaq 네오위즈홀딩스 *
가상화폐(비트코인 등) 331 kosdaq 에이티넘인베스트 *
가상화폐(비트코인 등) 331 kosdaq 아이오케이 *
가상화폐(비트코인 등) 331 kosdaq 클라우드에어 *
가상화폐(비트코인 등) 331 ko

KeyboardInterrupt: 